# ECEF to LLA to ECEF Coordinate Changer

This notebook was created by Sparrow Roch for use with the "Observatory Position Offset as a Source of Noise in Pulsar Timing Searches for Gravitational Waves" project.

This project analyses how data is affected based on simulated offsets in positional parameters for different observatories and telescopes (Green Bank Telescope and Arecibo Observatory, for now). Early stages in this project involved changing the x, y, and z geocentric (Earth-centered Earth-fixed (ECEF)) coordinates equally. However, it is believed, based on how pulsar signals are observed, that changes in observatory altitude may be most significant. Thus, to offset an observatory's position, one must...

1. Convert ECEF to latitude, longitude, and altitude (LLA)
2. Impose an altitude position offset
3. Convert back from LLA to ECEF with this new altitude offset

Preliminary results from earlier stages of the project indicate that offsets greater than 1000 m lead to unusable data. Offsets at 100 m (changed equally in ECEF x, y, and z) are dubious as well, but to a less-offensive extent. This notebook will be used to help generate new pulsar datasets with offsets ranging between 0 - 100 m.

In [10]:
import numpy as np
import matplotlib as plt
from astropy import units as u

# ECEF to LLA

TEMPO2 uses the Geodetic Reference System 1980 (GRS80) for
its geodetic coordinates (or, Earth-centered Earth-fixed (ECEF)
coordinates).

Especially when measuring altitudes, the Earth cannot be considered a perfect sphere. However, the geoid (Earth's topographical surface) is both complicated and dynamic, thus making it too irregular for point positioning. Instead, a reference ellipsoid matching the geoid volume is used as the surface for such calculations.

The 1980 Geodetic Reference System (GRS80) is one such reference ellipsoid. Many parameters go into the full GRS80 model. Here, only those necessary for this ECEF to LLA to ECEF calculation are included.

For now, please see the Wiki for fuller details:
https://en.wikipedia.org/wiki/Geodetic_Reference_System_1980

### GRS80 Constants

In [11]:
##############################
# Geometrical constants

# Semi-major axis (equatorial radius)
a = 6378137.0 * u.m
print('a: '+str(a))

# Flattening
# Note: this is a derived constant. Together, f and a can be
# used to derive all other necessary constants.
f = 0.003352810681183637418
print('f: '+str(f))

a: 6378137.0 m
f: 0.0033528106811836376


In [12]:
##############################
# Derived geometrical constants

# Semi-minor axis (polar radius)
# b = 6356752.314140347 * u.m
b = a * (1 - f)
print('b: '+str(b))

# (First) Eccentricity of elliptical section through poles
# e = ((a**2 - b**2)**(1/2))/a = 0.0818191910428
e = np.longdouble(((a**2 - b**2)**(1/2))/a)
print('e: '+str(e))

##############################
# Derived physical constants

# Second eccentricity = ((a**2 - b**2)**(1/2))/b
# also defined as e_prime = e/((1 - e**2)**(1/2)).
# (Either option yields same result.)
e_prime = np.longdouble(e/((1 - e**2)**(1/2)))
print('e_prime: '+str(e_prime))

b: 6356752.314140348 m
e: 0.081819191042831612704
e_prime: 0.08209443815193318474


## ECEF to LLA by closed formula set

TEMPO2's observatories.dat and PINT's observatories.py are
currently in agreement. PINT conveniently notes differences
in pre-2021 vs current data, and this data is also accessible in
archival versions of TEMPO2. There is no difference
in coords between TEMPO2 and PINT, only between pre-2021 to
current coords. These are included here for easy access.

Please see PINT documentation for explanations:
https://github.com/nanograv/PINT/blob/master/src/pint/observatory/observatories.py

In [13]:
# GBT (current):                 GBT (pre-2021):
#     X: 882589.289                  X: 882589.65
#     Y: -4924872.368                Y: -4924872.32
#     Z: 3943729.418                 Z: 3943729.348

# AO (current):                  AO (pre-2021):
#     X: 2390487.080                 X: 2390490.0
#     Y: -5564731.357                Y: -5564764.0
#     Z: 1994720.633                 Z: 1994727.0

# ECEF XYZ coords for your telescope or observatory of 
# choice. Unless stated otherwise, units are in meters.
X_original = 882589.289 * u.m
Y_original = -4924872.368 * u.m
Z_original = 3943729.418 * u.m

print('Starting with original ECEF coords:')
print('X: '+str(X_original))
print('Y: '+str(Y_original))
print('Z: '+str(Z_original))

Starting with original ECEF coords:
X: 882589.289 m
Y: -4924872.368 m
Z: 3943729.418 m


In [14]:
# I am using this reference for the conversion:
# https://docs.google.com/viewer?a=v&pid=sites&srcid=ZGVmYXVsdGRvbWFpbnxjdXJzb2RlZ2VvZGVzaWF8Z3g6MmE5ZDBlOWY2Y2MyNWZjZA
# Note that it uses WGS84 (instead of GRS80),
# but the calculations should be the same.

# Longitude (shortened lambda to lam)
lam = np.degrees(np.arctan(Y_original/X_original))
print('lam: '+str(lam))

# An auxiliary value (for convenience)
p = (X_original**2 + Y_original**2)**(1/2)
print('p: '+str(p))

# Another auxiliary value (for convenience)
theta = np.degrees(np.arctan((Z_original * a)/(p * b)))
print('theta: '+str(theta))

# Latitude
phi = np.degrees(np.arctan((Z_original + e_prime**2 * b * (np.sin(theta)**3))/(p - e**2 * a * (np.cos(theta)**3))))
print('phi: '+str(phi))

# Radius of curvature (in meters) defined as N.
# NOTE: Check how equatorial and polar radii of
# curvature differ.
N = a / ((1 - e**2 * (np.sin(phi)**2))**(1/2))
print('N: '+str(N))

# Height above ellipsoid in meters - this is the altitude
h = (p / np.cos(phi)) - N
print('h: '+str(h))

lam: -79.83984263406875 deg
p: 5003332.059152925 m
theta: 38.339494132743425 deg
phi: 38.433129637659945 deg
N: 6386401.962597307 m
h: 823.6680528549191 m


In [15]:
# Note that these results are consistent with those found on
# http://www.sysense.com/products/ecef_lla_converter/index.html

print('ECEF to LLA Results:')
print('Latitude: '+str(phi))
print('Longitude: '+str(lam))
print('Altitude: '+str(h))

ECEF to LLA Results:
Latitude: 38.433129637659945 deg
Longitude: -79.83984263406875 deg
Altitude: 823.6680528549191 m


# Imposing an Altitude Offset

In [16]:
# Altitude offset amount in meters
alt_offset = 0.05 * u.m
print('Altitude offset: '+str(alt_offset))

h_offset = h + alt_offset
print('Updated offset altitude: '+str(h_offset))

Altitude offset: 0.05 m
Updated offset altitude: 823.718052854919 m


In [17]:
print('Offset Results:')
print('Latitude: '+str(phi))
print('Longitude: '+str(lam))
print('Altitude: '+str(h_offset))

Offset Results:
Latitude: 38.433129637659945 deg
Longitude: -79.83984263406875 deg
Altitude: 823.718052854919 m


# LLA to ECEF

In [18]:
print('Offset ECEF Coordinates: ')

X_new = (N + h_offset) * np.cos(phi) * np.cos(lam)
print('X: '+str(X_new))

Y_new = (N + h_offset) * np.cos(phi) * np.sin(lam)
print('Y: '+str(Y_new))

Z_new = (((b**2 / a**2)*N) + h_offset) * np.sin(phi)
print('Z: '+str(Z_new))


Offset ECEF Coordinates: 
X: 882589.2959090199 m
Y: -4924872.406552516 m
Z: 3943729.4490800486 m


# Misc.

In [10]:
# Differences between original and offset coord values:
X_dif = X_new - X_original
print('X difference: '+str(X_dif))
Y_dif = Y_new - Y_original
print('Y difference: '+str(Y_dif))
Z_dif = Z_new - Z_original
print('Z difference: '+str(Z_dif))

X difference: 10.363528784965467 m
Y difference: -57.828774018589684 m
Z difference: 46.62006185455766 m


In [20]:
# NOTE: When including 0 m altitude offset, there are
# still small differences in X, Y, and Z. However, these
# are on the order of magnitude 10e-9 to 10e-10 meters.
# For pulsar timing, these are not significant.
#
# It seems to come from differences in using given vs
# calculated values, and related to how many significant
# digits are included in any calculations.